In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [9]:
prob = np.array([[0.6, 0.4],[0.3, 0.7]])

init = np.array([[0.5, 0.5]])

for n in range(10):
    init = init @ prob
    print(init)

    

[[0.45 0.55]]
[[0.435 0.565]]
[[0.4305 0.5695]]
[[0.42915 0.57085]]
[[0.428745 0.571255]]
[[0.4286235 0.5713765]]
[[0.42858705 0.57141295]]
[[0.42857611 0.57142388]]
[[0.42857283 0.57142717]]
[[0.42857185 0.57142815]]


In [24]:
beta = 1  # beta = 1 / (k_b * T)
T = 1 / beta

# particle properties
epsilon = 1,
sigma = 1 
mass = 1
density = 0.5
min_dist = 0.8

# MC algorithm params:
delta_rmax = 0.5
delta_Vmax = 0.5

n_dims = 3
box_size = np.array([8,8,8])
n_parts = int(density * np.prod(box_size))

# def Lennard_Jones(r, moved_part=None, dist_cutoff=3):
#     # r is a 2D array where row i, and col j correspond to particle i and dimension j (x:0, y:1, z:2)
#     # outputs: a 3D array where f[i,j,k] is the force on i because of j in dimension k,
#     # outputs a 2D array where u(i,j) is the potential between particles i and j
#     # sum of potential energy, and virial for instantaneous pressure calcs
    
#     #             part i    part j  (x,y,z)

# #     f = np.zeros([n_parts, n_parts, n_dims])
#     E_p_sum = 0
    
#     virial = 0    # for pressure calculation

#     for i in range(n_parts):
#         # get position vectors and distances between particles 
#         r_ij_vect = r[i,:] - r[(i+1):,:]   
#         r_ij_vect -= np.around(r_ij_vect / box_size) * box_size   #pbc
        
#         r_ij = np.sqrt(np.sum(r_ij_vect**2, axis=1))   # magnitude (norm) of r_ij_vect

#         # get index of particles within range of the spherical cutoff
#         parts_in_range = np.where(r_ij <= dist_cutoff)[0]
        
#         # distance magnitude of particles within the spherical cutoff
#         r_near = r_ij[parts_in_range]   
        
#         # Lennard Jones
#         # eq 5.3 in Allen and Tildesley 2nd ed
#         # fij = -du(rij)/dr * unit vector of r_ij_vect
#         f_mag = np.zeros([n_parts-i-1])
#         f_mag[parts_in_range] = 24 * epsilon / r_near * (2 *
#                 (sigma/r_near) ** 12 - (sigma/r_near) ** 6)
                
#         # new axis converts 1D array to a 2D array 
#         f_vect = f_mag[:, np.newaxis] * r_ij_vect/r_ij[:, np.newaxis]
# #         f[i, (i+1):, :] =  f_vect
# #         f[(i+1):, i, :] = -f_vect   # newton's 3rd law
        
        
#         virial += np.sum(r_ij_vect * f_vect)
        
#         Ep = np.zeros([n_parts - i - 1])
#         u[i, parts_in_range] = 4 * epsilon * ((sigma/r_near) ** 12 - (sigma/r_near) ** 6)
#         Ep[parts_in_range] = 4 * epsilon * ((sigma/r_near) ** 12 - (sigma/r_near) ** 6)
        
#         E_p_sum += np.sum(Ep)

#     virial = virial / 3
#     return f, E_p_sum, virial





init_locs = np.random.rand(n_parts, n_dims) * box_size
min_dists = []
for i in range(n_parts):
    continue_looping = True
    if i == 0:
        continue
    loop_count = 0
    while continue_looping == True:
        if loop_count > 10000:
            raise InterruptedError('Loop was interrupted')

        dr = init_locs[i,:] - init_locs[:i,:]
        dr = dr - np.around(dr / box_size) * box_size   # PBC 
        dist = np.sqrt(np.sum(dr**2, axis=1))

        if (dist < min_dist).any() == True:
            init_locs[i, :] = np.random.rand(1, n_dims) * box_size
        else:
            continue_looping = False
            min_dists.append(min(dist))

        loop_count += 1



def lennard_jones(r, u, w, part_moved=None):
    
    dist_cutoff = 3
    for i in range(n_parts):
        if part_moved is not None:
            i = part_moved 

        r_ij_vect = r[i,:] - r[(i+1):,:]

        r_ij_vect -= np.around(r_ij_vect / box_size) * box_size   #pbc
        
        r_ij = np.sqrt(np.sum(r_ij_vect**2, axis=1))   # magnitude (norm) of r_ij_vect

        # get index of particles within range of the spherical cutoff
        parts_in_range = np.where(r_ij <= dist_cutoff)[0]
        
        # distance magnitude of particles within the spherical cutoff
        r_near = r_ij[parts_in_range] 
        
        # reset row to zeros
        u[i, (i+1):] = np.zeros(n_parts - i - 1)
        w[i, (i + 1):] = np.zeros(n_parts - i - 1)
        
        sr_12 = (sigma / r_near) ** 12
        sr_6 = (sigma / r_near) ** 6

        u[i, (i + 1 + parts_in_range)] = (sr_12 - sr_6) * 4 * epsilon
        w[i, (i + 1 + parts_in_range)] = (sr_6 - 2 * sr_12) * 24 * epsilon 
        
        if part_moved is not None:
            break
            
    return u, w

part_locs = init_locs
u = np.zeros([n_parts, n_parts])   # potential
w = np.zeros([n_parts, n_parts])   # virial

u, w = lennard_jones(part_locs, u, w, part_moved=None)

for n in range(10):
    old_config = part_locs
    
    # chose random step:
    rand_num = np.random.rand()
    rand_num = 0.7
    if rand_num >= 0.5:
        # move random particle
        rand_part = int(np.random.rand() * n_parts)
        rand_vect = np.random.rand(n_dims)
        new_loc = part_locs[rand_part,:] + delta_rmax * (2 * rand_vect - 1) 
        
        # update loc if out of box
        new_loc = new_loc - np.floor(new_loc / box_size) * box_size
        
        new_locs = part_locs
        new_locs[rand_part,:] = new_loc
        
        # update energy and virial with new location:
        u_new, w_new = lennard_jones(new_locs, u, w, rand_part) 
        
        delta_u = np.sum(u_new[rand_part,:] - u[rand_part,:])
        print(delta_u)
        alpha = np.exp(-beta * delta_u)
        print(alpha)
        
    else:
        # volume move
        box_size_new = box_size + delta_Vmax * (2 * np.random.rand(n_dims) - 1)
        V_new = np.product(box_size_new)
        
        # update particle locations:
        part_locs_new = part_locs * box_size_new / box_size
    
        # calculate new potential and virial for all particles
        u_new, w_new = lennard_jones(part_locs_new, u, w, part_moved=None)
        
        delta_u = np.sum(u_new - u)
        virial = -np.sum(w_new) / 3
        P = n_parts / V_new / beta + virial/V_new
        
        alpha = np.exp(-beta * (delta_u + P * (V_new - V) - n_parts * (1/beta) * np.log(V_new / V)))
    
        
#     # accept or reject move?
#     alpha_choices = [alpha, 1]
#     prob = min(alpha_choices)
#     if prob < 1:
#         rand_num = np.random.rand()
#         if rand_num < 
    
    print(alpha)
    break
    

0.0
1.0
1.0


In [44]:
part_locs
print(np.shape(part_locs))
i = 0
dr = part_locs[i,:] - part_locs[(i+1):,:] 
dr -= np.around(dr / box_size) * box_size 
r_ij = np.sqrt(np.sum(dr**2, axis=1))
print(np.shape(r_ij))
r_ij[np.where(r_ij <= 3)[0]]


(256, 3)
(255,)


array([2.63629317, 2.97671321, 1.4767647 , 2.97580135, 2.75567902,
       2.31078983, 1.38428225, 1.31887321, 2.44838287, 1.27553513,
       2.91974039, 1.5171399 , 2.26557768, 2.37815481, 2.62322747,
       1.14701455, 2.57530043, 2.48526389, 2.86522746, 2.74893139,
       2.79790174, 2.55575077, 2.06163841, 2.51630649, 2.88869456,
       1.98102484, 2.17472427, 2.60359102, 1.32472495, 2.77465433,
       2.90019389, 2.74940079, 2.90018527, 2.616896  , 2.75389274,
       2.22424258, 2.01947872, 2.60266949, 2.42807214, 2.41003294,
       2.88283672, 2.14781851, 1.21543555, 1.62664323, 2.53952143,
       2.92294253, 2.3324331 , 2.95856852, 2.9614858 , 2.22504006,
       1.56004777, 1.28256025, 2.3396715 , 2.14330775])

In [32]:
box_size_new = box_size * 0.95
box_size_old = box_size
part_locs_new = box_size_new / box_size_old * part_locs
part_locs_new
part_locs_new.max(axis=0)

array([7.55239115, 7.57156234, 7.59008687])

In [30]:
box_size_new

array([7.6, 7.6, 7.6])